In [1]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import bs4 as bs
import re

import matplotlib.font_manager as font_manager
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter

In [2]:
#BAJAR INFORMACIÓN AUTOMÁTICAMENTE PARA UN CIK
#SE PUEDEN BUSCAR ACÁ -> https://www.sec.gov/edgar/searchedgar/cik.htm

headers = { 'User-Agent': 'Mozilla/5.0', } 

#Hacer un diccionario con emisores, CIKs, etc
# CIK = '0001326801' #-> Facebook
# CIK = '0001065280' #-> Netflix
# CIK = '0000789019' #-> Microsoft
# CIK = '0001318605' # -> Tesla
CIK = '0001018724' #-> Amazon
# CIK = '0001577552' #-> ALIBABA GROUP HOLDING LTD

   

#Bajar la información desde el sitio web de la fed y ponerlo en formato json
download = requests.get('https://data.sec.gov/api/xbrl/companyfacts/CIK'+CIK+'.json', stream=True, headers = headers)
fin_statment = download.json()

#Sacar los facts según US_GAAP o IFRS
us_gaap_concepts = list(fin_statment['facts']['us-gaap'].keys())
# ifrs_concepts = list(fin_statment['facts']['ifrs-full'].keys())

us_gaap_concepts

['AccountsPayable',
 'AccountsPayableCurrent',
 'AccountsReceivableNetCurrent',
 'AccruedLiabilities',
 'AccruedLiabilitiesCurrent',
 'AccruedLiabilitiesForUnredeeemedGiftCards',
 'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment',
 'AccumulatedOtherComprehensiveIncomeLossAvailableForSaleSecuritiesAdjustmentNetOfTax',
 'AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax',
 'AccumulatedOtherComprehensiveIncomeLossNetOfTax',
 'AcquiredFiniteLivedIntangibleAssetWeightedAverageUsefulLife',
 'AdditionalPaidInCapital',
 'AdjustmentsNoncashItemsToReconcileNetIncomeLossToCashProvidedByUsedInOperatingActivitiesOther',
 'AdjustmentsToAdditionalPaidInCapitalTaxEffectFromShareBasedCompensation',
 'AdvertisingExpense',
 'AllocatedShareBasedCompensationExpense',
 'AllowanceForDoubtfulAccountsReceivable',
 'AllowanceForDoubtfulAccountsReceivableCurrent',
 'AllowanceForDoubtfulAccountsReceivableWriteOffs',
 'AmortizationOfAcquiredIntangibleAsset

In [29]:
#Assets
Assets_list = fin_statment['facts']['us-gaap']['Assets']['units']['USD']
Assets_df = pd.DataFrame(Assets_list)
Assets_df

,end,val,accn,fy,fp,form,filed,frame
0,2008-12-31,8314000000,0001193125-09-154174,2009,Q2,10-Q,2009-07-24,NaN
1,2008-12-31,8314000000,0001193125-09-212134,2009,Q3,10-Q,2009-10-23,NaN
2,2008-12-31,8314000000,0001193125-10-016098,2009,FY,10-K,2010-01-29,CY2008Q4I
3,2009-06-30,7675000000,0001193125-09-154174,2009,Q2,10-Q,2009-07-24,CY2009Q2I
4,2009-09-30,8972000000,0001193125-09-212134,2009,Q3,10-Q,2009-10-23,CY2009Q3I
...,...,...,...,...,...,...,...,...
104,2020-12-31,321195000000,0001018724-22-000005,2021,FY,10-K,2022-02-04,CY2020Q4I
105,2021-03-31,323077000000,0001018724-21-000010,2021,Q1,10-Q,2021-04-30,CY2021Q1I
106,2021-06-30,360319000000,0001018724-21-000020,2021,Q2,10-Q,2021-07-30,CY2021Q2I
107,2021-09-30,382406000000,0001018724-21-000028,2021,Q3,10-Q,2021-10-29,CY2021Q3I


In [7]:
#AssetsCurrent
AssetsCurrent_list = fin_statment['facts']['us-gaap']['AssetsCurrent']['units']['USD']
AssetsCurrent_df = pd.DataFrame(AssetsCurrent_list)
AssetsCurrent_df

,end,val,accn,fy,fp,form,filed,frame
0,2008-12-31,6157000000,0001193125-09-154174,2009,Q2,10-Q,2009-07-24,NaN
1,2008-12-31,6157000000,0001193125-09-212134,2009,Q3,10-Q,2009-10-23,NaN
2,2008-12-31,6157000000,0001193125-10-016098,2009,FY,10-K,2010-01-29,CY2008Q4I
3,2009-06-30,5304000000,0001193125-09-154174,2009,Q2,10-Q,2009-07-24,CY2009Q2I
4,2009-09-30,6369000000,0001193125-09-212134,2009,Q3,10-Q,2009-10-23,CY2009Q3I
...,...,...,...,...,...,...,...,...
97,2020-12-31,132733000000,0001018724-22-000005,2021,FY,10-K,2022-02-04,CY2020Q4I
98,2021-03-31,121408000000,0001018724-21-000010,2021,Q1,10-Q,2021-04-30,CY2021Q1I
99,2021-06-30,140848000000,0001018724-21-000020,2021,Q2,10-Q,2021-07-30,CY2021Q2I
100,2021-09-30,138531000000,0001018724-21-000028,2021,Q3,10-Q,2021-10-29,CY2021Q3I


In [11]:
#LiabilitiesCurrent
LiabilitiesCurrent_list = fin_statment['facts']['us-gaap']['LiabilitiesCurrent']['units']['USD']
LiabilitiesCurrent_df = pd.DataFrame(LiabilitiesCurrent_list)
LiabilitiesCurrent_df

,end,val,accn,fy,fp,form,filed,frame
0,2008-12-31,4746000000,0001193125-09-154174,2009,Q2,10-Q,2009-07-24,NaN
1,2008-12-31,4746000000,0001193125-09-212134,2009,Q3,10-Q,2009-10-23,NaN
2,2008-12-31,4746000000,0001193125-10-016098,2009,FY,10-K,2010-01-29,CY2008Q4I
3,2009-06-30,3636000000,0001193125-09-154174,2009,Q2,10-Q,2009-07-24,CY2009Q2I
4,2009-09-30,4537000000,0001193125-09-212134,2009,Q3,10-Q,2009-10-23,CY2009Q3I
...,...,...,...,...,...,...,...,...
97,2020-12-31,126385000000,0001018724-22-000005,2021,FY,10-K,2022-02-04,CY2020Q4I
98,2021-03-31,115404000000,0001018724-21-000010,2021,Q1,10-Q,2021-04-30,CY2021Q1I
99,2021-06-30,117792000000,0001018724-21-000020,2021,Q2,10-Q,2021-07-30,CY2021Q2I
100,2021-09-30,123994000000,0001018724-21-000028,2021,Q3,10-Q,2021-10-29,CY2021Q3I


In [36]:
#OtherLiabilitiesNoncurrent
LiabilitiesNonCurrent_list = fin_statment['facts']['us-gaap']['OtherLiabilitiesNoncurrent']['units']['USD']
LiabilitiesNonCurrent_df = pd.DataFrame(LiabilitiesNonCurrent_list)
LiabilitiesNonCurrent_df

,end,val,accn,fy,fp,form,filed,frame
0,2008-12-31,487000000,0001193125-09-154174,2009,Q2,10-Q,2009-07-24,NaN
1,2008-12-31,487000000,0001193125-09-212134,2009,Q3,10-Q,2009-10-23,NaN
2,2008-12-31,487000000,0001193125-10-016098,2009,FY,10-K,2010-01-29,CY2008Q4I
3,2009-06-30,674000000,0001193125-09-154174,2009,Q2,10-Q,2009-07-24,CY2009Q2I
4,2009-09-30,734000000,0001193125-09-212134,2009,Q3,10-Q,2009-10-23,CY2009Q3I
...,...,...,...,...,...,...,...,...
85,2020-12-31,17017000000,0001018724-22-000005,2021,FY,10-K,2022-02-04,CY2020Q4I
86,2021-03-31,19418000000,0001018724-21-000010,2021,Q1,10-Q,2021-04-30,CY2021Q1I
87,2021-06-30,21148000000,0001018724-21-000020,2021,Q2,10-Q,2021-07-30,CY2021Q2I
88,2021-09-30,23945000000,0001018724-21-000028,2021,Q3,10-Q,2021-10-29,CY2021Q3I


In [16]:
#LiabilitiesAndStockholdersEquity
LiabilitiesAndStockholdersEquity_list = fin_statment['facts']['us-gaap']['LiabilitiesAndStockholdersEquity']['units']['USD']
LiabilitiesAndStockholdersEquity_df = pd.DataFrame(LiabilitiesAndStockholdersEquity_list)
LiabilitiesAndStockholdersEquity_df

,end,val,accn,fy,fp,form,filed,frame
0,2008-12-31,8314000000,0001193125-09-154174,2009,Q2,10-Q,2009-07-24,NaN
1,2008-12-31,8314000000,0001193125-09-212134,2009,Q3,10-Q,2009-10-23,NaN
2,2008-12-31,8314000000,0001193125-10-016098,2009,FY,10-K,2010-01-29,CY2008Q4I
3,2009-06-30,7675000000,0001193125-09-154174,2009,Q2,10-Q,2009-07-24,CY2009Q2I
4,2009-09-30,8972000000,0001193125-09-212134,2009,Q3,10-Q,2009-10-23,CY2009Q3I
...,...,...,...,...,...,...,...,...
97,2020-12-31,321195000000,0001018724-22-000005,2021,FY,10-K,2022-02-04,CY2020Q4I
98,2021-03-31,323077000000,0001018724-21-000010,2021,Q1,10-Q,2021-04-30,CY2021Q1I
99,2021-06-30,360319000000,0001018724-21-000020,2021,Q2,10-Q,2021-07-30,CY2021Q2I
100,2021-09-30,382406000000,0001018724-21-000028,2021,Q3,10-Q,2021-10-29,CY2021Q3I


In [15]:
#LiabilitiesNonCurrent
StockholdersEquity_list = fin_statment['facts']['us-gaap']['StockholdersEquity']['units']['USD']
StockholdersEquity_df = pd.DataFrame(StockholdersEquity_list)
StockholdersEquity_df

,end,val,accn,fy,fp,form,filed,frame
0,2006-12-31,431000000,0001193125-10-016098,2009,FY,10-K,2010-01-29,CY2006Q4I
1,2007-12-31,1197000000,0001193125-10-016098,2009,FY,10-K,2010-01-29,NaN
2,2007-12-31,1197000000,0001193125-11-016253,2010,FY,10-K,2011-01-28,CY2007Q4I
3,2008-12-31,2672000000,0001193125-09-154174,2009,Q2,10-Q,2009-07-24,NaN
4,2008-12-31,2672000000,0001193125-09-212134,2009,Q3,10-Q,2009-10-23,NaN
...,...,...,...,...,...,...,...,...
153,2021-03-31,103320000000,0001018724-21-000020,2021,Q2,10-Q,2021-07-30,CY2021Q1I
154,2021-06-30,114803000000,0001018724-21-000020,2021,Q2,10-Q,2021-07-30,NaN
155,2021-06-30,114803000000,0001018724-21-000028,2021,Q3,10-Q,2021-10-29,CY2021Q2I
156,2021-09-30,120564000000,0001018724-21-000028,2021,Q3,10-Q,2021-10-29,CY2021Q3I


In [5]:
#NetIncomeLoss
NetIncomeLoss_list = fin_statment['facts']['us-gaap']['NetIncomeLoss']['units']['USD']
NetIncomeLoss_df = pd.DataFrame(NetIncomeLoss_list)
NetIncomeLoss_df

,start,end,val,accn,fy,fp,form,filed,frame
0,2007-01-01,2007-12-31,476000000,0001193125-10-016098,2009,FY,10-K,2010-01-29,CY2007
1,2007-07-01,2008-06-30,588000000,0001193125-09-154174,2009,Q2,10-Q,2009-07-24,NaN
2,2008-01-01,2008-06-30,301000000,0001193125-09-154174,2009,Q2,10-Q,2009-07-24,NaN
3,2008-04-01,2008-06-30,158000000,0001193125-09-154174,2009,Q2,10-Q,2009-07-24,CY2008Q2
4,2007-10-01,2008-09-30,627000000,0001193125-09-212134,2009,Q3,10-Q,2009-10-23,NaN
...,...,...,...,...,...,...,...,...,...
326,2021-04-01,2021-06-30,7778000000,0001018724-21-000020,2021,Q2,10-Q,2021-07-30,CY2021Q2
327,2020-10-01,2021-09-30,26263000000,0001018724-21-000028,2021,Q3,10-Q,2021-10-29,NaN
328,2021-01-01,2021-09-30,19041000000,0001018724-21-000028,2021,Q3,10-Q,2021-10-29,NaN
329,2021-07-01,2021-09-30,3156000000,0001018724-21-000028,2021,Q3,10-Q,2021-10-29,CY2021Q3


In [8]:
#DepreciationAndAmortization
DepreciationAndAmortization_list = fin_statment['facts']['us-gaap']['DepreciationAndAmortization']['units']['USD']
DepreciationAndAmortization_df = pd.DataFrame(DepreciationAndAmortization_list)
DepreciationAndAmortization_df

,start,end,val,accn,fy,fp,form,filed,frame
0,2007-01-01,2007-12-31,246000000,0001193125-10-016098,2009,FY,10-K,2010-01-29,CY2007
1,2007-07-01,2008-06-30,259000000,0001193125-09-154174,2009,Q2,10-Q,2009-07-24,NaN
2,2008-01-01,2008-06-30,134000000,0001193125-09-154174,2009,Q2,10-Q,2009-07-24,NaN
3,2008-04-01,2008-06-30,70000000,0001193125-09-154174,2009,Q2,10-Q,2009-07-24,CY2008Q2
4,2007-10-01,2008-09-30,273000000,0001193125-09-212134,2009,Q3,10-Q,2009-10-23,NaN
...,...,...,...,...,...,...,...,...,...
147,2016-01-01,2016-09-30,5819000000,0001018724-16-000324,2016,Q3,10-Q,2016-10-28,NaN
148,2016-07-01,2016-09-30,2084000000,0001018724-16-000324,2016,Q3,10-Q,2016-10-28,CY2016Q3
149,2016-01-01,2016-12-31,8116000000,0001018724-17-000011,2016,FY,10-K,2017-02-10,CY2016
150,2016-04-01,2017-03-31,8725000000,0001018724-17-000051,2017,Q1,10-Q,2017-04-28,NaN


In [9]:
fin_statment_df['start'] = pd.to_datetime(fin_statment_df['start'])
fin_statment_df['end'] = pd.to_datetime(fin_statment_df['end'])
fin_statment_df['dias'] = (fin_statment_df['end']-fin_statment_df['start']).dt.days



# new_row = {'start':datetime(2021,9,30), 
#            'end':datetime(2021,12,31), 
#            'val':14323000000, 
#            'accn':'AAA', 
#            'fy':'AAA', 
#            'fp':'AAA', 
#            'form':'AAA', 
#            'filed':'AAA',            
#            'frame':'CY',            
#            'dias':90}
# fin_statment_df = fin_statment_df.append(new_row, ignore_index=True)

# fin_statment_df['start'].iloc[-1] = datetime(2021,9,30)
# fin_statment_df['end'].iloc[-1] = datetime(2021,12,31)
# fin_statment_df['val'].iloc[-1] = 17719000000
# fin_statment_df['frame'].iloc[-1] = 'CY'
# fin_statment_df['dias'].iloc[-1] = 90


accounts_to_plot = fin_statment_df.dropna()
accounts_to_plot = accounts_to_plot[accounts_to_plot['frame'].str.contains('CY')]
# accounts_to_plot = accounts_to_plot[accounts_to_plot['dias']<100]
accounts_to_plot

,start,end,val,accn,fy,fp,form,filed,frame,dias
0,2014-04-01,2015-03-31,3914000000,0001047469-15-005768,2014,FY,20-F,2015-06-25,CY2014,364
1,2015-04-01,2016-03-31,11083000000,0001047469-16-013400,2015,FY,20-F,2016-05-24,CY2015,365
2,2016-04-01,2017-03-31,6345000000,0001047469-17-007286,2016,FY,6-K,2017-11-24,CY2016,364
4,2017-04-01,2018-03-31,10218000000,0001047469-18-005257,2017,FY,20-F,2018-07-27,CY2017,364
5,2018-04-01,2019-03-31,13095000000,0001047469-19-006257,2019,FY,6-K,2019-11-13,CY2018,364
7,2019-04-01,2020-03-31,21104000000,0001104659-21-010913,2020,FY,6-K,2021-02-02,CY2019,365
10,2020-04-01,2021-03-31,22983000000,0001104659-21-096092,2021,FY,20-F,2021-07-27,CY2020,364
